In [ ]:
!pip install onnxruntime-gpu onnx  optimum evaluate datasets transformers sentencepiece accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from copy import deepcopy
from onnxruntime import InferenceSession
from onnxruntime.transformers.optimizer import optimize_model
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = 'cuda'

In [ ]:
from datasets import load_dataset
dataset = load_dataset("morzecrew/RefinedPersonaChat")
# dataset = load_dataset("SiberiaSoft/SiberianPersonaChat")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
with open('toxic.txt', 'r') as file:
    answer_toxic = [i.strip() for i in file.readlines()]

In [ ]:
import numpy as np
np.random.choice(answer_toxic)

'Я понимаю, что мы можем иметь разные точки зрения, но позвольте нам сосредоточиться на поиске общих идей.'

In [ ]:
import multiprocessing as mp
from tqdm.notebook import tqdm

n_workers = 2 * mp.cpu_count()

In [ ]:
df_train = dataset['train']

In [ ]:
df_train[0]

{'output': 'Пахра — река Москвы и Московской области, правый приток Москвы-реки. Длина реки составляет 135 км, площадь водосборного бассейна — 2580 км². Исток Пахры находится в 3,8 км северо-западнее станции Бекасово-1 Киевского направления и Большого кольца Московской железной дороги, в болотах Наро-Фоминского района. Река впадает в Москву-реку рядом с посёлком Мячково Раменского района. Ширина поймы — 100—200 метров, русла в среднем течении 15—20 метров, в низовьях — 30—40 метров. Русло реки извилистое. По характеру водного режима Пахра относится к волжскому типу, основным источником питания являются снеговые воды. Летом и осенью реку питают дождевые и подземные воды, зимой — родники. Половодье начинается с конца марта, заканчивается в начале мая. Подъём воды в реке достигает 6 метров. Пахра замерзает в ноябре — декабре, оттаивает в конце марта — апреля. Среднегодовой расход воды в 36 км от устья — 9,95 м³/с, что соответствует объёму стока 0,314 км³/год. По химическому составу вода П

In [ ]:
len(df_train)

In [ ]:
model_checkpoint = 'morzecrew/rubert-tiny-toxicity-onnx-optimized-fp16'#'cointegrated/rubert-tiny-toxicity'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint,use_io_binding=True ).to(device)

The ONNX file optimized_fp16.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


In [ ]:
from torch.nn.utils import prune
pruner = prune.L1Unstructured(amount=0.05)
state_dict = model.state_dict()

for key in state_dict.keys():
    if "weight" in key:
        state_dict[key] = pruner.prune(state_dict[key])
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
if torch.cuda.is_available():
    model.cuda()

In [ ]:
import torch

model = torch.quantization.quantize_dynamic(
    model=model,
    qconfig_spec = {torch.nn.Linear: torch.quantization.default_dynamic_qconfig}, dtype=torch.qint8)

In [ ]:
from transformers import pipeline

model_checkpoint = 'morzecrew/rubert-base-cased-nli-threeway-onnx-optimized-fp16'#'cointegrated/rubert-tiny-toxicity'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
onnx_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
onnx_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint,use_io_binding=True ).to(device)

classifier = pipeline("zero-shot-classification", model = onnx_model, tokenizer=onnx_tokenizer)

The ONNX file optimized_fp16.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


In [ ]:
import numpy as np


def textprocessing(dct):

    candidate_labels = ["политика", "не политика"]
    text = dct['input'] + dct['output']

    politic = classifier(text, candidate_labels, multi_label=False)
    politic_proba = politic['scores'][politic['labels'].index('политика')]

    if politic_proba > 0.9:
        return np.nan

    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()

    proba = proba[0]
    proba = 1 - proba.T[0] * (1 - max(proba.T[1:4]))

    if proba > 0.8:
        dct['output'] = np.random.choice(answer_toxic)
    return dct

In [ ]:
%%time
p = mp.Pool(n_workers)

data_temp = np.array(p.map(textprocessing, tqdm(df_train[:5])))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 110 ms, sys: 174 ms, total: 284 ms
Wall time: 14.4 s


In [ ]:
import pandas as pd

data_temp = data_temp[~pd.isnull(data_temp)]

In [ ]:
data_temp.shape

(4,)

In [ ]:
import json

with open("data_not_toxic_politic.json", "w", encoding='utf-8') as json_file:
    json.dump({'train': list(data_temp)}, json_file)